In [1]:
import numpy as np
import pandas as pd
from ortools.graph import pywrapgraph
from collections import defaultdict, Counter

In [2]:
sub = pd.read_csv('drop_t390_t260.csv').values.tolist()

In [3]:
triplets = 5001
twins = 45001
wish = pd.read_csv('Data/child_wishlist_v2.csv', header=None).as_matrix()[:twins, 1:]

In [4]:
counts = Counter(elem[1] for elem in sub)

gifts_remain = defaultdict(int)
for i,j in counts.items():
    if j != 1000 and i != -1:
        gifts_remain[i] = 1000 - j
    if i == -1:
        print i, j
print gifts_remain

-1 4188
defaultdict(<type 'int'>, {320L: 560, 272L: 253, 389L: 223, 998L: 467, 494L: 993, 240L: 247, 118L: 691, 186L: 10, 671L: 744})


In [5]:
def fix_twins(score):
    for child in range(triplets,twins,2):
        gifts = set()
        gift_happy = defaultdict(int)
        if sub[child][1] == -1:
            for j in range(2):
                child2 = child + j
                for index in range(100):
                    gift = wish[child2][index]
                    gifts.add(gift)
                    gift_happy[gift] += (100 - index)*2

            gift_add = []
            for gift in gifts:
                if gift in gifts_remain.keys():
                    gift_add.append(gift)

            if gift_add:            
#                print child, gift_add#, gift_happy[gift]
                for gift in gift_add:
                    if gifts_remain[gift] >= 4 and gift_happy[gift] > score:
                        print "Twins",child, gift, gift_happy[gift]
                        sub[child][1] = gift
                        sub[child+1][1] = gift
                        gifts_remain[gift] -= 2

In [6]:
def fix_triplets(score):
    for child in range(0,triplets,3):
        gifts = set()
        gift_happy = defaultdict(int)
        if sub[child][1] == -1:
            for j in range(3):
                child3 = child + j
                for index in range(100):
                    gift = wish[child3][index]
                    gifts.add(gift)
                    gift_happy[gift] += (100 - index)*2

            gift_add = []
            for gift in gifts:
                if gift in gifts_remain.keys():
                    gift_add.append(gift)

            if gift_add:            
#                print child, gift_add#, gift_happy[gift]
                for gift in gift_add:
                    if (gifts_remain[gift] == 3 or gifts_remain[gift]>=5) and gift_happy[gift] > score:
                        print "Triplet",child, gift, gift_happy[gift]
                        sub[child][1] = gift
                        sub[child+1][1] = gift
                        sub[child+2][1] = gift
                        gifts_remain[gift] -= 3

In [7]:
for i in range(200,0,-20):
    print "Score", i
    fix_twins(i)
    fix_triplets(i)

Score 200
Score 180
Twins 16145 240 194
Triplet 102 320 188
Triplet 4467 118 188
Score 160
Triplet 84 389 164
Triplet 2631 186 180
Triplet 2682 389 170
Triplet 4053 272 172
Triplet 4356 671 178
Score 140
Triplet 915 186 156
Triplet 2814 389 152
Score 120
Twins 5671 186 138
Twins 28389 389 124
Triplet 834 272 122
Triplet 2247 998 138
Triplet 2517 118 128
Score 100
Triplet 2691 998 112
Triplet 4728 240 112
Score 80
Twins 13561 998 86
Twins 28205 240 86
Triplet 2022 389 90
Triplet 3864 320 90
Score 60
Triplet 732 240 74
Triplet 786 240 68
Triplet 1713 272 70
Triplet 3399 118 76
Triplet 3528 320 74
Triplet 4458 118 70
Score 40
Triplet 2145 320 56
Score 20
Triplet 2505 240 38


In [9]:
print gifts_remain

defaultdict(<type 'int'>, {320L: 548, 272L: 244, 389L: 209, 998L: 459, 494L: 993, 240L: 231, 118L: 679, 186L: 2, 671L: 741})


In [8]:
print "Add remained gift to childs"

Add remained gift to childs


In [9]:
Ntwins = 0
for child in range(triplets,twins,2):
    if sub[child][1] == -1:
        Ntwins += 1
        
Ntriplets = 0
for child in range(0,triplets,3):
    if sub[child][1] == -1:
        Ntriplets += 1

print "Ntwins", Ntwins
print "Ntriplets", Ntriplets

Ntwins 532
Ntriplets 1014


In [10]:
gifts = gifts_remain.keys()
n2 = defaultdict(int); n3 = defaultdict(int)
for gift in gifts:
    if gifts_remain[gift] % 6 == 1:
        gifts_remain[gift] -= 7
        Ntwins -= 2
        n2[gift] = 2
        Ntriplets -= 1
        n3[gift] = 1
    elif gifts_remain[gift] % 6 == 2:
        gifts_remain[gift] -= 2
        Ntwins -= 1
        n2[gift] = 1
    elif gifts_remain[gift] % 6 == 3:
        gifts_remain[gift] -= 3
        Ntriplets -= 1
        n3[gift] = 1
    elif gifts_remain[gift] % 6 == 4:
        gifts_remain[gift] -= 4
        Ntwins -= 2
        n2[gift] = 2
    elif gifts_remain[gift] % 6 == 5:
        gifts_remain[gift] -= 5
        Ntwins -= 1
        n2[gift] = 1
        Ntriplets -= 1
        n3[gift] = 1
print gifts_remain

defaultdict(<type 'int'>, {320L: 546, 272L: 240, 389L: 204, 998L: 456, 494L: 990, 240L: 228, 118L: 672, 186L: 0, 671L: 738})


In [11]:
print Ntwins, Ntriplets

525 1008


In [12]:
print n2
print n3

defaultdict(<type 'int'>, {320L: 1, 272L: 2, 186L: 1, 389L: 1, 118L: 2})
defaultdict(<type 'int'>, {389L: 1, 998L: 1, 494L: 1, 240L: 1, 118L: 1, 671L: 1})


In [13]:
for i in range(0,Ntwins/3):
    for gift in gifts:
        if gifts_remain[gift] > 0 :
            gifts_remain[gift] -= 6
            n2[gift] += 3
            break

In [14]:
print gifts_remain

defaultdict(<type 'int'>, {320L: 0, 272L: 0, 389L: 0, 998L: 396, 494L: 990, 240L: 228, 118L: 672, 186L: 0, 671L: 738})


In [15]:
for i in range(0,Ntriplets/2):
    for gift in gifts:
        if gifts_remain[gift] > 0 :
            gifts_remain[gift] -= 6
            n3[gift] += 2
            break

In [16]:
print gifts_remain

defaultdict(<type 'int'>, {320L: 0, 272L: 0, 389L: 0, 998L: 0, 494L: 0, 240L: 0, 118L: 0, 186L: 0, 671L: 0})


In [17]:
print n2
print n3

defaultdict(<type 'int'>, {320L: 274, 389L: 103, 998L: 30, 272L: 122, 118L: 2, 186L: 1})
defaultdict(<type 'int'>, {389L: 1, 998L: 133, 494L: 331, 240L: 77, 118L: 225, 671L: 247})


In [18]:
counts = Counter(elem[1] for elem in sub)

gifts_remain = defaultdict(int)
for i,j in counts.items():
    if j != 1000 and i != -1:
        gifts_remain[i] = 1000 - j
    if i == -1:
        print i, j
print gifts_remain

-1 4106
defaultdict(<type 'int'>, {320: 548, 272: 244, 389: 209, 998: 459, 494L: 993, 240: 231, 118: 679, 186: 2, 671: 741})


In [19]:
# Twins
gifts = n2.keys()
for child in range(triplets,twins,2):
    if sub[child][1] == -1:
        for gift in gifts:
            if n2[gift]>0 :
                sub[child][1] = gift
                sub[child+1][1] = gift
                gifts_remain[gift] -= 2
                n2[gift] -= 1
                break

In [20]:
# Triplets
gifts = n3.keys()
for child in range(0,triplets,3):
    if sub[child][1] == -1:
        for gift in gifts:
            if (gifts_remain[gift] == 3 or gifts_remain[gift]>=5):
                sub[child][1] = gift
                sub[child+1][1] = gift
                sub[child+2][1] = gift
                gifts_remain[gift] -= 3
                n3[gift] -= 1
                break

In [21]:
print gifts_remain

defaultdict(<type 'int'>, {320: 0, 272: 0, 389: 0, 998: 0, 494L: 0, 240: 0, 118: 0, 186: 0, 671: 0})


In [22]:
counts = Counter(elem[1] for elem in sub)

gifts = defaultdict(int)
for i,j in counts.items():
    if j != 1000 and i != -1:
        gifts[i] = j
    if i == -1:
        print "There is a problem", i, j
print gifts

defaultdict(<type 'int'>, {})


In [23]:
out = open('drop_t390_t260_fix5_2.csv', 'w')
out.write('ChildId,GiftId\n')
for i in range(len(sub)):
    out.write(str(sub[i][0]) + ',' + str(sub[i][1]) + '\n')
out.close()